<a href="https://colab.research.google.com/github/dlwogns/ML-DL/blob/main/GoDaddy%20-%20Microbusiness%20Density%20Forecasting/GoDaddy_Dense256_refactoring_by_function.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

### train

cflips - 연방정보처리 과정에 따른 각각의 카운티의 고유 넘버. 앞 두자리는 주(state), 뒤 세자리는 카운티. <br>
first day of month - 달의 첫날.<br>
microbusiness density - 각 카운티의 100명당 microbusiness 지수. 이 지표에 사용된 인구수는 2년씩 지연되어있음. (21년 density는 19년의 인구를 사용.) <br>
active - 각 카운티의 microbusiness raw count를 나타냄.<br>

### sample_submission
target -> microbusiness density<br>

### test

row_id, cfips, first day of month 만 제공. -> 시계열

### census starter
관련한 정보들 모음.<br>
이것도 또한 2년의 지연이 있음.<br>
pct_bb -> 카운티 인터넷가입 가정 비율 <br>
pct_college -> 인구수 대비 25세 이상 4년제 대학 학생 비율.<br>

pct foreign born -> 미국 외에서 태어난 사람 비율. 인구대비<br>
pct it workers -> 노동자 중 it산업 종사자 비율<br>
median hh inc -> 카운티 가장 수입 중앙값

# aim to
19년 1월 ~ 22년 10월까지의 정보가 주어지고, 22년 11월 ~ 23년 1월까지 예측

# reference
https://www.kaggle.com/code/kimtaehun/complete-baseline-code-with-various-ml-model<br>
baseline https://www.kaggle.com/code/masahiroichigo/godaddy-microbusiness-density-forecasting-with-tf<br>

In [123]:
train = pd.read_csv("/content/drive/MyDrive/data/GoDaddy/train.csv")
test = pd.read_csv("/content/drive/MyDrive/data/GoDaddy/test.csv")
census = pd.read_csv("/content/drive/MyDrive/data/GoDaddy/census_starter.csv")
sample_submission = pd.read_csv("/content/drive/MyDrive/data/GoDaddy/sample_submission.csv")

In [3]:
train.nunique()

row_id                   122265
cfips                      3135
county                     1871
state                        51
first_day_of_month           39
microbusiness_density     97122
active                    19193
dtype: int64

In [4]:
test.nunique()

row_id                25080
cfips                  3135
first_day_of_month        8
dtype: int64

39개의 데이터를 받아서, 8개를 나타내야됨.

census에는 있지만, train에는 없는 것을 제외

In [5]:
cen = census['cfips'].unique()
tra = train['cfips'].unique()
extra = []
for c in cen:
  if c not in tra:
    extra.append(c)
extra

[2158, 2261, 31117, 46102, 51580, 51640, 51690]

In [6]:
extra_index = []
for c in extra:
  idx = census[census['cfips'] == c].index[0]
  extra_index.append(idx)
census.iloc[extra_index]

,pct_bb_2017,pct_bb_2018,pct_bb_2019,pct_bb_2020,pct_bb_2021,cfips,pct_college_2017,pct_college_2018,pct_college_2019,pct_college_2020,...,pct_it_workers_2017,pct_it_workers_2018,pct_it_workers_2019,pct_it_workers_2020,pct_it_workers_2021,median_hh_inc_2017,median_hh_inc_2018,median_hh_inc_2019,median_hh_inc_2020,median_hh_inc_2021
81,58.4,62.5,65.3,66.1,70.0,2158,3.0,3.3,2.8,2.6,...,1.7,1.9,1.2,1.1,0.9,36468,35539.0,36754,37358.0,37975.0
92,80.5,79.1,80.4,NaN,NaN,2261,23.1,19.0,16.5,NaN,...,3.3,3.9,5.3,NaN,NaN,86019,82306.0,79867,NaN,NaN
1712,62.0,59.6,62.8,68.8,78.6,31117,19.6,18.0,17.6,18.5,...,3.8,0.0,0.0,0.0,0.0,60714,48882.0,48125,51932.0,53750.0
2412,44.8,48.5,45.2,53.0,56.5,46102,9.5,8.7,7.3,6.8,...,0.5,0.6,0.0,0.0,0.0,27804,30347.0,31997,31423.0,33192.0
2921,59.8,69.0,77.2,80.1,82.9,51580,8.7,10.3,10.4,10.3,...,2.0,1.9,1.6,0.4,0.4,35546,39432.0,40655,41024.0,41242.0
2928,63.1,66.8,68.5,75.4,81.7,51640,5.7,7.4,8.3,9.7,...,1.1,0.6,1.5,0.8,0.8,31311,31938.0,33575,35184.0,39808.0
2936,63.4,72.1,73.9,76.6,80.2,51690,13.1,14.1,14.2,14.5,...,1.4,1.7,1.7,1.6,0.3,33545,33224.0,34371,36166.0,36832.0


In [7]:
census.drop(extra_index, axis=0, inplace = True)
len(census)

3135

In [8]:
pd.merge(train, census, how='inner', on='cfips')
# on은 기준열을 잡아주는 것으로, default는 None이고 이 경우에는 index를 기준으로 한다.

,row_id,cfips,county,state,first_day_of_month,microbusiness_density,active,pct_bb_2017,pct_bb_2018,pct_bb_2019,...,pct_it_workers_2017,pct_it_workers_2018,pct_it_workers_2019,pct_it_workers_2020,pct_it_workers_2021,median_hh_inc_2017,median_hh_inc_2018,median_hh_inc_2019,median_hh_inc_2020,median_hh_inc_2021
0,1001_2019-08-01,1001,Autauga County,Alabama,2019-08-01,3.007682,1249,76.6,78.9,80.6,...,1.3,1.1,0.7,0.6,1.1,55317,58786.0,58731,57982.0,62660.0
1,1001_2019-09-01,1001,Autauga County,Alabama,2019-09-01,2.884870,1198,76.6,78.9,80.6,...,1.3,1.1,0.7,0.6,1.1,55317,58786.0,58731,57982.0,62660.0
2,1001_2019-10-01,1001,Autauga County,Alabama,2019-10-01,3.055843,1269,76.6,78.9,80.6,...,1.3,1.1,0.7,0.6,1.1,55317,58786.0,58731,57982.0,62660.0
3,1001_2019-11-01,1001,Autauga County,Alabama,2019-11-01,2.993233,1243,76.6,78.9,80.6,...,1.3,1.1,0.7,0.6,1.1,55317,58786.0,58731,57982.0,62660.0
4,1001_2019-12-01,1001,Autauga County,Alabama,2019-12-01,2.993233,1243,76.6,78.9,80.6,...,1.3,1.1,0.7,0.6,1.1,55317,58786.0,58731,57982.0,62660.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122260,56045_2022-06-01,56045,Weston County,Wyoming,2022-06-01,1.803249,101,71.1,73.3,76.8,...,0.6,0.6,0.0,0.0,0.0,59605,52867.0,57031,53333.0,65566.0
122261,56045_2022-07-01,56045,Weston County,Wyoming,2022-07-01,1.803249,101,71.1,73.3,76.8,...,0.6,0.6,0.0,0.0,0.0,59605,52867.0,57031,53333.0,65566.0
122262,56045_2022-08-01,56045,Weston County,Wyoming,2022-08-01,1.785395,100,71.1,73.3,76.8,...,0.6,0.6,0.0,0.0,0.0,59605,52867.0,57031,53333.0,65566.0
122263,56045_2022-09-01,56045,Weston County,Wyoming,2022-09-01,1.785395,100,71.1,73.3,76.8,...,0.6,0.6,0.0,0.0,0.0,59605,52867.0,57031,53333.0,65566.0


In [124]:
train["first_day_of_month"] = pd.to_datetime(train["first_day_of_month"])
train = train.sort_values(["cfips", "first_day_of_month"]).reset_index(drop = True)

test["first_day_of_month"] = pd.to_datetime(test["first_day_of_month"])
test = test.sort_values(["cfips", "first_day_of_month"]).reset_index(drop = True)

In [10]:
train

,row_id,cfips,county,state,first_day_of_month,microbusiness_density,active
0,1001_2019-08-01,1001,Autauga County,Alabama,2019-08-01,3.007682,1249
1,1001_2019-09-01,1001,Autauga County,Alabama,2019-09-01,2.884870,1198
2,1001_2019-10-01,1001,Autauga County,Alabama,2019-10-01,3.055843,1269
3,1001_2019-11-01,1001,Autauga County,Alabama,2019-11-01,2.993233,1243
4,1001_2019-12-01,1001,Autauga County,Alabama,2019-12-01,2.993233,1243
...,...,...,...,...,...,...,...
122260,56045_2022-06-01,56045,Weston County,Wyoming,2022-06-01,1.803249,101
122261,56045_2022-07-01,56045,Weston County,Wyoming,2022-07-01,1.803249,101
122262,56045_2022-08-01,56045,Weston County,Wyoming,2022-08-01,1.785395,100
122263,56045_2022-09-01,56045,Weston County,Wyoming,2022-09-01,1.785395,100


In [11]:
import tensorflow as tf

### Time Series Windowing Data
https://albertum.medium.com/preprocessing-time-series-to-windowed-datasets-a464799b1df7

In [34]:
HORIZON = 8
WINDOW = 8

def get_labelled_window(x, horizon = HORIZON):
  return x[:, :-horizon], x[:, -horizon:]

def make_windows(x, window_size=WINDOW, horizon = HORIZON):
  window_step = np.expand_dims(np.arange(window_size+horizon), axis=0)
  # arange -> 0 ~ argument까지의 일정한 step의 1차원 배열 return.
  # expand_dims -> [1,2], axis=0 -> [[1,2]] [1,2], axis=1 -> [[1],[2]]
  # 차원을 확대하는 것
  # time series를 위한 windowing data를 만들기 위한 
  window_indexes = window_step + np.expand_dims(np.arange(len(x) - (window_size + horizon-1)), axis=0).T
  #print(f'Window indexes: \n{window_indexes, window_indexes.shape}')

  windowed_array = x[window_indexes]

  windows, labels = get_labelled_window(windowed_array, horizon = horizon)

  return windows, labels

In [13]:
def make_train_test_split(windows, labels, split_size=1):
  split_size = len(windows) - split_size
  train_windows = windows[:split_size]
  train_labels = labels[:split_size]
  test_windows = windows[split_size:]
  test_labels = labels[split_size:]
  return train_windows, test_windows, train_labels, test_labels

In [14]:
def evaluate_preds(y_true, y_pred):
  y_true = tf.cast(y_true, dtype = tf.float32)
  y_pred = tf.cast(y_pred, dtype = tf.float32)
  mae = tf.keras.metrics.mean_absolute_error(y_true, y_pred)[0]
  mse = tf.keras.metrics.mean_squared_error(y_true, y_pred)[0]
  rmse = tf.sqrt(mse)
  mape = tf.keras.metrics.mean_absolute_percentage_error(y_true, y_pred)[0]

  return {"mae" : mae.numpy(),
          "mse" : mse.numpy(),
          "rmse" : rmse.numpy(),
          "mape" : mape.numpy()}

In [15]:
def make_preds(model, input_data):
  forecast = model.predict(input_data)
  return tf.squeeze(forecast)

In [16]:
def plot_time_series(cfips, timesteps, values, format="-", start=0, end=None, label=None):
    """
    Plots timesteps (a series of points in time) against values (a series of values across timesteps).

    Parameters
    -----------
    timesteps : array of timesteps values
    values : array of values across time
    format : style of plot, default "."
    start : where to start the plot (setting a value will index from start of timesteps & values)
    end : where to end the plot (similar to start but for the end)
    label : label to show on plot about values
    """
    # Plot the series
    plt.plot(timesteps[start:end], values[start:end], format, label=label)
    plt.title(f'Cfips: {cfips}', fontsize=18)
    plt.xlabel("time", fontsize=14)
    plt.ylabel('MicroBusiness Density', fontsize=14)
    if label:
        plt.legend(fontsize=14) # make label bigger
    plt.grid(True)

In [21]:
IDS = train.cfips.unique()
IDS

array([ 1001,  1003,  1005, ..., 56041, 56043, 56045])

random data to check model

In [22]:
for cfips in IDS[:5]:
  print(cfips)

1001
1003
1005
1007
1009


In [27]:
IDSRANDOM = np.random.choice(IDS, 20)

In [28]:
x_train = np.arange(39).reshape((-1,1))
x_test = np.arange(31, 39).reshape((-1,1))
## for ploting

In [93]:
forecasts = []
test_result_mse = {}
test_result_mape = {}

In [121]:
# Original code
# mape = []
# for cfips in IDSRANDOM:
#   cur_cfip = cfips
#   cur_train = train.loc[train.cfips == cur_cfip]
#   last = cur_train.microbusiness_density.values[-1]
#   # last?

#   windows, labels = make_windows(cur_train.microbusiness_density.values)
#   train_windows, test_windows, train_labels, test_labels = make_train_test_split(windows, labels)

#   print(train_windows.shape, train_labels.shape)
#   model = tf.keras.Sequential()
#   model.add(tf.keras.layers.Lambda(lambda x : tf.expand_dims(x, axis=1)))
#   model.add(tf.keras.layers.Dense(128, activation="tanh"))
#   model.add(tf.keras.layers.Dense(8))

#   model.compile(loss='mape', optimizer = tf.keras.optimizers.Adam(), metrics=['mape', 'mse'])

#   history = model.fit(x=train_windows, y=train_labels, epochs=100, verbose = 0)

#   preds = make_preds(model, test_windows)

#   results = evaluate_preds(test_labels, preds)
#   m = results['mape']
#   mape.append(m)

#   plot_time_series(cur_cfip, x_train, cur_train.microbusiness_density.values, label='train_data')
#   plot_time_series(cur_cfip, x_test, preds, label='forecast')
#   plt.show()


def test_model(layer, layer_name):
  print("TEST FOR", layer_name)
  print("###############################################################################")
  rmse = []
  mape = []
  for cfips in IDSRANDOM:
    cur_cfip = cfips
    cur_train = train.loc[train.cfips == cur_cfip]
    last = cur_train.microbusiness_density.values[-1]
    # last?

    windows, labels = make_windows(cur_train.microbusiness_density.values)
    train_windows, test_windows, train_labels, test_labels = make_train_test_split(windows, labels)

    #print(train_windows.shape, train_labels.shape)
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Lambda(lambda x : tf.expand_dims(x, axis=1)))
    #model.add(tf.keras.layers.Dense(128, activation=activation))
    model.add(layer)
    model.add(tf.keras.layers.Dense(8))

    model.compile(loss='mse', optimizer = tf.keras.optimizers.Adam(), metrics=['mse'])

    history = model.fit(x=train_windows, y=train_labels, epochs=100, verbose = 0)

    preds = make_preds(model, test_windows)

    results = evaluate_preds(test_labels, preds)

    rm = results['rmse']
    ma = results['mape']
    rmse.append(rm)
    mape.append(ma)

    # plot_time_series(cur_cfip, x_train, cur_train.microbusiness_density.values, label='train_data')
    # plot_time_series(cur_cfip, x_test, preds, label='forecast')
    # plt.show()
  test_result_mse[layer_name] = np.array(rmse).sum()/len(rmse)
  test_result_mape[layer_name] = np.array(mape).sum()/len(mape)
  print("###############################################################################")

  return np.array(rmse).sum()/len(rmse), np.array(mape).sum()/len(mape)


In [99]:
test_model(tf.keras.layers.Dense(64, activation="tanh"),"Dense 64 tanh")
test_model(tf.keras.layers.Dense(64, activation = "relu"), "Dense 64 relu")
test_model(tf.keras.layers.Dense(128, activation="tanh"), "Dense 128 tanh")
test_model(tf.keras.layers.Dense(128, activation = "relu"), "Dense 128 relu")
test_model(tf.keras.layers.Dense(256, activation="tanh"), "Dense 256 tanh")
test_model(tf.keras.layers.Dense(256, activation = "relu"), "Dense 256 relu")
test_model(tf.keras.layers.LSTM(32, activation="tanh"), "LSTM 32 tanh")
test_model(tf.keras.layers.LSTM(64, activation="tanh"),"LSTM 64 tanh")
test_model(tf.keras.layers.LSTM(128, activation="tanh"),"LSTM 128 tanh")

TEST FOR Dense 64 tanh
###############################################################################
(23, 8) (23, 8)
1/1 [==============================] - 0s 80ms/step
(23, 8) (23, 8)
1/1 [==============================] - 0s 79ms/step
(23, 8) (23, 8)
1/1 [==============================] - 0s 75ms/step
(23, 8) (23, 8)
1/1 [==============================] - 0s 76ms/step
(23, 8) (23, 8)
1/1 [==============================] - 0s 81ms/step
(23, 8) (23, 8)
1/1 [==============================] - 0s 74ms/step
(23, 8) (23, 8)
1/1 [==============================] - 0s 73ms/step
(23, 8) (23, 8)
1/1 [==============================] - 0s 122ms/step
(23, 8) (23, 8)
1/1 [==============================] - 0s 105ms/step
(23, 8) (23, 8)
1/1 [==============================] - 0s 110ms/step
(23, 8) (23, 8)
1/1 [==============================] - 0s 72ms/step
(23, 8) (23, 8)
1/1 [==============================] - 0s 74ms/step
(23, 8) (23, 8)
1/1 [==============================] - 0s 71ms/step
(23, 8) (2

(0.15049588680267334, 5.712225723266601)

In [101]:
test_result_mse

{'Dense 64 tanh': 0.20985381603240966,
 'Dense 64 relu': 0.38098940849304197,
 'Dense 128 tanh': 0.16152970790863036,
 'Dense 128 relu': 0.17994199991226195,
 'Dense 256 tanh': 0.15030664205551147,
 'Dense 256 relu': 0.1732219934463501,
 'LSTM 32 tanh': 0.8651004791259765,
 'LSTM 64 tanh': 0.3052075386047363,
 'LSTM 128 tanh': 0.15049588680267334}

In [102]:
test_result_mape

{'Dense 64 tanh': 6.878441619873047,
 'Dense 64 relu': 11.482707214355468,
 'Dense 128 tanh': 5.700801849365234,
 'Dense 128 relu': 6.573223876953125,
 'Dense 256 tanh': 5.533488464355469,
 'Dense 256 relu': 6.282940673828125,
 'LSTM 32 tanh': 24.780284118652343,
 'LSTM 64 tanh': 8.507736206054688,
 'LSTM 128 tanh': 5.712225723266601}

Final

In [106]:
IDS = train.cfips.unique()
x_train = np.arange(39).reshape((-1,1))
x_test = np.arange(39, 47).reshape((-1,1))

mapes = []
forecasts = []

HORIZON = 8
WINDOW = 8

for cfips in IDS:
  c = cfips
  df = train.loc[train.cfips == c]
  last = df.microbusiness_density.values[-1]

  windows, labels = make_windows(df.microbusiness_density.values)

  model = tf.keras.Sequential()
  model.add(tf.keras.layers.Lambda(lambda x : tf.expand_dims(x, axis=1)))
  model.add(tf.keras.layers.Dense(256, activation = "tanh"))
  model.add(tf.keras.layers.Dense(8))

  model.compile(loss = 'mae', optimizer = tf.keras.optimizers.Adam(), metrics=['mae', 'mse'])

  model.fit(x=windows, y = labels, epochs = 100, batch_size = 256, verbose = 0)

  forecast = model.predict(tf.expand_dims(labels[-1], axis=0))
  shift = last - np.squeeze(forecast)[0]
  preds = np.squeeze(forecast) + shift
  forecasts.append(preds)

  # plot_time_series(c, x_train, df.microbusiness_density.values, label='raw')
  # plot_time_series(c, x_test, preds, label='forecast')
  # plt.show()

  print(forecasts[-1])


스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
[2.4791663 2.418011  2.3853307 2.4601946 2.3809428 2.3850636 2.3546762
 2.386108 ]
1/1 [==============================] - 0s 76ms/step
[0.6596021  0.6612829  0.6430499  0.66711646 0.66397107 0.6862335
 0.68715435 0.6897448 ]
1/1 [==============================] - 0s 73ms/step
[2.1800232 2.151895  2.184377  2.2030156 2.1895068 2.2396533 2.2376082
 2.227579 ]
1/1 [==============================] - 0s 76ms/step
[1.0639656 1.0568595 1.0725845 1.0703286 1.0821023 1.0734662 1.0853724
 1.0797143]
1/1 [==============================] - 0s 73ms/step
[1.2171323 1.2216935 1.2128893 1.2091931 1.2406093 1.2058641 1.2375298
 1.2219156]
1/1 [==============================] - 0s 71ms/step
[0.62928003 0.62048405 0.6153792  0.6051596  0.54336184 0.49804783
 0.49115646 0.46560818]
1/1 [==============================] - 0s 71ms/step
[4.1906366 4.1892166 4.2045126 4.2048864 4.228152  4.18923   4.2048626
 4.197961 ]
1/1 [==============================] - 0s 115ms/step
[0.

In [107]:
pd.DataFrame(forecasts).to_csv('forecast.csv', index=False)

In [108]:
t = forecasts

In [109]:
t = np.array(t).reshape((-1))

In [116]:
type(forecasts)

list

In [125]:
test

,row_id,cfips,first_day_of_month
0,1001_2022-11-01,1001,2022-11-01
1,1001_2022-12-01,1001,2022-12-01
2,1001_2023-01-01,1001,2023-01-01
3,1001_2023-02-01,1001,2023-02-01
4,1001_2023-03-01,1001,2023-03-01
...,...,...,...
25075,56045_2023-02-01,56045,2023-02-01
25076,56045_2023-03-01,56045,2023-03-01
25077,56045_2023-04-01,56045,2023-04-01
25078,56045_2023-05-01,56045,2023-05-01


In [126]:
test['microbusiness_density'] = np.array(forecasts).reshape((-1))

sub = test[['row_id', 'microbusiness_density']]
sub.to_csv('submission.csv', index=False)
print(sub.shape)
sub.head()

(25080, 2)


,row_id,microbusiness_density
0,1001_2022-11-01,3.463856
1,1001_2022-12-01,3.517263
2,1001_2023-01-01,3.496802
3,1001_2023-02-01,3.459977
4,1001_2023-03-01,3.514518
